In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

In [ ]:
# import kbrgan
# import kbrgan.kernel as kernel
# import kbrgan.main as main
# import kbrgan.embed as embed
# import kbrgan.util as util

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
import pymc3 as pm

## Examples from the introduction

http://docs.pymc.io/intro.html

In [ ]:
# Data
n = np.ones(4)*5
y = np.array([0, 1, 3, 5])
dose = np.array([-.86,-.3,-.05,.73])

with pm.Model() as bioassay_model:

    # Prior distributions for latent variables
    alpha = pm.Normal('alpha', 0, sd=100)
    beta = pm.Normal('beta', 0, sd=100)

    # Linear combinations of parameters
    theta = pm.invlogit(alpha + beta*dose)

    # Model likelihood
    deaths = pm.Binomial('deaths', n=n, p=theta, observed=y)

In [ ]:
bioassay_model

In [ ]:
with bioassay_model:

    # Draw wamples
    trace = pm.sample(1000, njobs=3)
    # Plot two parameters
    pm.forestplot(trace, varnames=['alpha', 'beta'])

## Linear regression

http://docs.pymc.io/notebooks/getting_started.html#Abstract

\begin{split}\begin{aligned}
Y  &\sim \mathcal{N}(\mu, \sigma^2) \\
\mu &= \alpha + \beta_1 X_1 + \beta_2 X_2
\end{aligned}\end{split}

Half-normal prior for $\sigma$:

\begin{split}\begin{aligned}
\alpha &\sim \mathcal{N}(0, 100) \\
\beta_i &\sim \mathcal{N}(0, 100) \\
\sigma &\sim \lvert\mathcal{N}(0, 1){\rvert}
\end{aligned}\end{split}

In [ ]:
# generate data

# Initialize random number generator
np.random.seed(124)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,4))
axes[0].scatter(X1, Y)
axes[1].scatter(X2, Y)
axes[0].set_ylabel('Y'); axes[0].set_xlabel('X1'); axes[1].set_xlabel('X2');

In [ ]:
# model
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10, shape=2)
    sigma = pm.HalfNormal('sigma', sd=1)

    # Expected value of outcome
    mu = alpha + beta[0]*X1 + beta[1]*X2

    # Likelihood (sampling distribution) of observations
    # Observe Y (from above)
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=Y)


In [ ]:
# MAP estimate
map_estimate = pm.find_MAP(model=basic_model)
map_estimate

In [ ]:
# sampling from the posterior
with basic_model:
    # sample 500 points
    trace = pm.sample(500)

In [ ]:
plt.plot(trace['alpha'][-100:])
plt.ylabel(r'$\alpha$')

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.summary(trace).round(2)